## **Scraping Develpoers Data from TopCoders using Public API & Selnium.**
#### **Flow & Working of Code:**
 - Get Challenges:
    - Active Challenges [**Block-1**]
    - Past Challenges   [**Block-2**]
    - We are going to extract developers profiles from these challenges.
 - Get Developers RAW Data:
    - Provide the path of Challenges Json File [**Block-4**].
 - Get Developers Traits & Statictics:
    - RAW Developers Data File Used As Input [**Block-5**]
 - Convert the Developers Profiles Json into CSV:
    - Developers Profiles File Used As Input [**Block-6**]

In [ ]:
%pip install requests
%pip install tqdm
%pip install undetected_chromedriver

import os
import csv
import sys
import time
import json
import datetime
import requests
from tqdm import tqdm
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [4]:
CURRENT_DIR = os.getcwd()
currentTime = datetime.datetime.now()
WORKING_SESSION = os.path.join(CURRENT_DIR, f"SESSION_{currentTime.day}_{currentTime.hour}_{currentTime.minute}")
BIN_DIR = os.path.join(WORKING_SESSION, "bin")
LOG_DIR = os.path.join(WORKING_SESSION, "log")
LOG_FILE = os.path.join(LOG_DIR, "log.json")

if not os.path.exists(WORKING_SESSION): os.makedirs(WORKING_SESSION)
if not os.path.exists(BIN_DIR): os.makedirs(BIN_DIR)
if not os.path.exists(LOG_DIR): os.makedirs(LOG_DIR)
if not os.path.exists(LOG_FILE): 
    with open(LOG_FILE, "w") as f: f.write(json.dumps({}))


url = "https://api.topcoder.com/v5/challenges/"
headers = {
    "accept": "*/*",
    "accept-language": "en-US,en;q=0.9",
    "app-version": "1.1.0",
    "content-type": "application/json",
    "if-none-match": "W/\"14029-03jo/8tgikL7RQIliQ90l5HcP4g\"",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Chromium\";v=\"124\", \"Microsoft Edge\";v=\"124\", \"Not-A.Brand\";v=\"99\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-site",
    "Referer": "https://www.topcoder.com/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
}
active_challenge_params = {
    "status": "Active",
    "perPage": "50",
    "page": "10",
    "sortBy": "startDate",
    "sortOrder": "desc",
    "tracks[]": ["Dev", "Des", "DS", "QA"],
    "types[]": ["CH", "F2F", "MM", "TSK"]
}
past_challenge_params = active_challenge_params
past_challenge_params['status'] = "Completed"
past_challenge_params['perPage'] = '100'
past_challenge_params['page'] = '1'

### Scrap Active Challenges from TopCoders

In [5]:

ACTIVE_CHALLENGES_FILE = os.path.join(BIN_DIR, "activeChallenges.json")
if not os.path.exists(ACTIVE_CHALLENGES_FILE): 
    with open(ACTIVE_CHALLENGES_FILE, "w") as f: f.write(json.dumps({}))

with open(LOG_FILE, "r") as f: log = json.loads(f.read())
try:
    with open(ACTIVE_CHALLENGES_FILE, "r") as f: activeChallenges = json.loads(f.read())
    if not activeChallenges: activeChallenges = []
except: activeChallenges = []


os.system('cls')
print(f"{'#'*50}")
print(f"SCRAP TOPCODERS Active CHALLENGES")
print(f"{'#'*50}\n")

startPages = int(input("Page to Start From: "))
endPages = int(input("Ending Limit of Page: "))

if 'activeChallengeScan' in log:
    if int(startPages)>=log['activeChallengeScan'][0] and int(startPages) <= log['activeChallengeScan'][1]:
        print(f"\nWe have Found You have Made A Request Recently Of {log['activeChallengeScan']} Pages.")
        print(f"Please Make Sure, If the Data Is In Current Dir, It Will Cause Duplication Of Data.")
        print(f"Its Up To You, To Start From Stretch Or Start The Pages Scan From Last Scan.\n")
        choice = input("Enter new page value or go with above (y/n): ")
        if choice.lower() in ['y', 'yes']:
            os.system('cls')
            startPages = int(input("Page to Start From: "))
            endPages = int(input("Ending Limit of Page: "))
            print("")
        else: pass


for i in tqdm(range(startPages, endPages+1)):
    active_challenge_params['page'] = f"{i}"
    response = requests.get(url, headers=headers, params=active_challenge_params).json()
    for chl in response: 
        if "id" in chl.keys(): activeChallenges.append(chl)


log['activeChallengeScan'] = [startPages, endPages]
with open(LOG_FILE, "w") as f: log = f.write(json.dumps(log))
with open(ACTIVE_CHALLENGES_FILE, "w") as f: f.write(json.dumps(activeChallenges))
print(f"\n {len(activeChallenges)} Challenges Found.\n DATA SAVED LOCATION: '{ACTIVE_CHALLENGES_FILE}'")

##################################################
SCRAP TOPCODERS Active CHALLENGES
##################################################



100%|██████████| 1/1 [00:03<00:00,  3.73s/it]


 100 Challenges Found.
 DATA SAVED LOCATION: 'c:\Users\PMLS\Desktop\topCoders\topCoders\SESSION_24_6_49\bin\activeChallenges.json'


### Scrap Past Completed Challenges from TopCoders

In [6]:
PAST_CHALLENGES_FILE = os.path.join(BIN_DIR, "pastChallenges.json")
if not os.path.exists(PAST_CHALLENGES_FILE): 
    with open(PAST_CHALLENGES_FILE, "w") as f: f.write(json.dumps({}))

os.system('cls')
print(f"{'#'*50}")
print(f"SCRAP TOPCODERS PAST CHALLENGES")
print(f"{'#'*50}\n")

startPages = int(input("Page to Start From: "))
endPages = int(input("Ending Limit of Page: "))


with open(LOG_FILE, "r") as f: log = json.loads(f.read())
try:
    with open(PAST_CHALLENGES_FILE, "r") as f: pastChallenges = json.loads(f.read())
    if not pastChallenges: pastChallenges = []
except: pastChallenges = []

if 'pastChallengeScan' in log:
    if int(startPages)>=log['pastChallengeScan'][0] and int(startPages) <= log['pastChallengeScan'][1]:
        print(f"\nWe have Found You have Made A Request Recently Of {log['pastChallengeScan']} Pages.")
        print(f"Please Make Sure, If the Data Is In Current Dir, It Will Cause Duplication Of Data.")
        print(f"Its Up To You, To Start From Stretch Or Start The Pages Scan From Last Scan.\n")
        choice = input("Enter new page value or go with above (y/n): ")
        if choice.lower() in ['y', 'yes']:
            os.system('cls')
            startPages = int(input("Page to Start From: "))
            endPages = int(input("Ending Limit of Page: "))
            print("")
        else: pass


for i in tqdm(range(startPages, endPages+1)):
    past_challenge_params['page'] = f"{i}"
    response = requests.get(url, headers=headers, params=past_challenge_params).json()
    for chl in response: 
        if "id" in chl.keys(): pastChallenges.append(chl)

with open(PAST_CHALLENGES_FILE, "w") as f: f.write(json.dumps(pastChallenges))
print(f"\n {len(pastChallenges)} Challenges Found...\n DATA SAVED: '{PAST_CHALLENGES_FILE}'")

log['pastChallengeScan'] = [startPages, endPages]
with open(LOG_FILE, "w") as f: log = f.write(json.dumps(log))

##################################################
SCRAP TOPCODERS PAST CHALLENGES
##################################################



100%|██████████| 1/1 [00:03<00:00,  3.35s/it]


 100 Challenges Found...
 DATA SAVED: 'c:\Users\PMLS\Desktop\topCoders\topCoders\SESSION_24_6_49\bin\pastChallenges.json'


In [7]:
class CHROME_DRIVER:
    def __init__(self):
        self.RUNNING = False
        self.Initialize_driver()
    
    def Initialize_driver(self):
        try:
            chrome_options = uc.ChromeOptions()
            chrome_options.headless = False
            chrome_options.arguments.extend(["--no-sandbox", "--disable-setuid-sandbox"])
            chrome_options.add_argument('--no-sandbox')
            chrome_options.add_argument('--disable-dev-shm-usage')
            chrome_options.add_argument("--incognito")
            chrome_options.add_argument('--blink-settings=imagesEnabled=false')
            chrome_options.add_argument('--disable-application-cache')
            chrome_options.add_argument('--disable-cache')
            chrome_options.add_argument('--disable-component-update')
            chrome_options.add_argument('--disable-extensions')
            chrome_options.add_argument('--disable-default-apps')
            chrome_options.add_argument("--disable-backgrounding-occluded-windows")
            chrome_options.add_argument("--start-minimized")
            chrome_options.add_argument('--disable-features=OptimizationGuideModelDownloading,OptimizationHintsFetching,OptimizationTargetPrediction,OptimizationHints')
            self.driver = uc.Chrome(options=chrome_options)
            time.sleep(2)
            self.RUNNING = True
        except Exception as e: print(e)
        
    def getDriver(self, url):
        if not self.RUNNING:  self.Initialize_driver()
        while True:
            try:
                self.driver.get(url)
                time.sleep(1)
                break
            except Exception as e:
                print(e)
                self.Initialize_driver()
                continue
        return self.driver

### SCRAP DEVELOPERS DATA USING SELENIUM

In [ ]:

headers['Referer'] = "https://profiles.topcoder.com/"

DEVELOPERS_FILE = os.path.join(BIN_DIR, "developers.json")
if not os.path.exists(DEVELOPERS_FILE): 
    with open(DEVELOPERS_FILE, "w") as f: f.write(json.dumps([{}]))

def readDevData():
    readedData = None
    with open(DEVELOPERS_FILE, "r") as f: readedData = json.loads(f.read())
    return readedData

def saveDevData(data): 
    with open(DEVELOPERS_FILE, "w") as f: f.write(json.dumps(data))
    
    
fileName = input("Enter Challenges File Path: ")
try:
    CHALLENGES = None
    with open(r"{}".format(fileName), "r") as f: CHALLENGES = json.loads(f.read())

    USER_DATA = readDevData()
    print(f"Got {len(USER_DATA[0])} Dev")
    WEBDRIVER = CHROME_DRIVER()
    driver = None
    for index, chl in enumerate(tqdm(CHALLENGES, desc="Processing challenges")):
        try: 
            url = f"https://www.topcoder.com/challenges/{chl['id']}?tab=registrants"
            driver = WEBDRIVER.getDriver(url)
            WebDriverWait(driver, timeout=10).until(EC.presence_of_all_elements_located((By.XPATH, "//*[@aria-label='Registrants']//a")))
            developers = driver.find_elements(By.XPATH, "//*[@aria-label='Registrants']//a")
            for _, dev in enumerate(developers):
                url = dev.get_attribute('href')
                username = dev.text
                developersData = {'username':username, "url":url}
                USER_DATA[0][username]={username: developersData}
                saveDevData(USER_DATA)  
        except Exception as e:
            print(e)
            driver.quit()
            continue
    saveDevData(USER_DATA)
    print(f"{index}/{len(CHALLENGES)} DEV Found: {len(developers)}")
    print(f"\n DEVELOPERS RAW DATA SAVED: \n'{DEVELOPERS_FILE}'")
    driver.quit()
except Exception as e: print(e)


### GET DEVELOPERS TRAITS AND STATISTICS.

In [ ]:
DEVELOPERS_PROFILE_FILE = os.path.join(BIN_DIR, "developerProfiles.json")
if not os.path.exists(DEVELOPERS_PROFILE_FILE): 
    with open(DEVELOPERS_PROFILE_FILE, "w") as f: f.write(json.dumps([{}]))

def getDeveloper(userName):
    url = f"https://api.topcoder.com/v5/members/{userName}/stats"
    global headers
    try:
        response = requests.get(url, headers=headers).json()
        return response
    except Exception as e: 
        print(e)
        return None

def devTraits(username, traitType="work"):
    url = f"https://api.topcoder.com/v5/members/{username}/traits?traitIds={traitType}"
    global headers
    try:
        response = requests.get(url, headers=headers).json()
        if response:
            traits = response[0]['traits']
            if traits['data']: return traits['data']
            return {}
    except Exception as e: 
        print(e)
        return {}
        
def statsAnalysis(devData):
    temp = {}
    for ky, vlu in devData.items():
        if type(vlu) == dict:
            if "challenges" in vlu.keys(): temp[ky] = {'challenges': vlu['challenges'], 'wins': vlu['wins']}
            elif ky=="maxRating":
                temp['rating'] = vlu['rating']
                try: temp['track'] = vlu['track']
                except: pass
            else: temp[ky] = vlu
    return temp

def readDevData(path):
    readedData = None
    with open(path, "r") as f:
        readedData = json.loads(f.read())
    return readedData

def saveDevData(data):
    with open(DEVELOPERS_PROFILE_FILE, "w") as f: f.write(json.dumps(data))


DEVELOPERS = readDevData(DEVELOPERS_FILE)
STATS = readDevData(DEVELOPERS_PROFILE_FILE)
keys = list(DEVELOPERS[0].keys())
for index, user in enumerate(tqdm(keys, desc="Processing challenges")):
    try:
        devData = getDeveloper(user)
        devStats = statsAnalysis(devData[0])
        workTraits = devTraits(user, "work")
        educationTraits = devTraits(user, "education")
        temp = {'username':user}
        temp.update(devStats)
        temp.update({"work": workTraits})
        temp.update({"eduction": educationTraits})
        STATS[0][user] = temp
        saveDevData(STATS)
        # print(f"{index}/{len(keys)}")
    except Exception as e:
        if "Failed to resolve 'api.topcoder.com' ([Errno 11001] getaddrinfo failed)" in str(e):
            input("Network Problem, Please Reconnect and Press Enter ...")
            continue
        print(e)


### CONVERT DEVELOPERS JSON FILE NTO CSV FILE

In [10]:
try: filePath = DEVELOPERS_PROFILE_FILE
except: 
    print("Please Input Path of File in CMD: py convertStats_toCSV.py filePath")
    sys.exit()

def readDevData(path):
    readedData = None
    with open(path, "r") as f:
        readedData = json.loads(f.read())
    return readedData

data = readDevData(r"{}".format(filePath))
usernames = set(k for k in data[0].keys())
currentTime = datetime.datetime.now()
csv_file_path = os.path.join(WORKING_SESSION, f"DEVELOPERS_STATS_{currentTime.day}_{currentTime.hour}_{currentTime.minute}.csv") 

with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['username', 'url', 'rating', 'track', 'Develop Skill', 'Design Skill', 'Data Science Skill', 'Copilot',
                        'Work Company 1', 'Work Industory 1', 'Work Position 1', 'Work cityTown 1', 'Work Status 1',
                        'Work Company 2', 'Work Industory 2', 'Work Position 2', 'Work cityTown 2', 'Work Status 2',
                        'Work Company 3', 'Work Industory 3', 'Work Position 3', 'Work cityTown 3', 'Work Status 3', 
                        'Education Major 1', 'Education School 1', 'Education Gratuated 1',
                        'Education Major 2', 'Education School 2', 'Education Gratuated 2',
                        'Education Major 3', 'Education School 3', 'Education Gratuated 3',
                        ])  # Write header

    for i in data[0].values():
        username = i['username']
        url = 'https://www.topcoder.com/members/'+i['username']
        rating = i['rating']
        try: track = i['track']
        except: track = None
        developSkill = 'challenges: ' + str(i['DEVELOP']['challenges']) + ' challenges: ' + str(i['DEVELOP']['wins'])
        designSkill = 'challenges: ' + str(i['DESIGN']['challenges']) + ' challenges: ' + str(i['DESIGN']['wins'])
        dataScienceSkill = 'challenges: ' + str(i['DATA_SCIENCE']['challenges']) + ' challenges: ' + str(i['DATA_SCIENCE']['wins'])
        try: CopilitSkill = '  '.join([f"{ky}: {val}" for ky, val in i['COPILOT'].items()])
        except: CopilitSkill = None

        if i['work']:
            workCompany = [x['company'] for x in i['work']]
            try: workIndustry = [x['industry'] for x in i['work']]
            except: workIndustry = [None for x in range(0, len(workCompany))]
            try: workPosition = [x['position'] for x in i['work']]
            except: workPosition = [None for x in range(0, len(workCompany))]
            try: workCityTowny = [x['cityTown'] for x in i['work']]
            except: workCityTowny = [None for x in range(0, len(workCompany))]
            try: workStatus = [x['working'] for x in i['work']]
            except: workStatus = [None for x in range(0, len(workCompany))]
        else:  workCompany = workIndustry = workPosition = workCityTowny = workStatus = []

        workCompany1, workCompany2, workCompany3 = None, None, None
        workIndustry1, workIndustry2, workIndustry3 = None, None, None
        workPosition1, workPosition2, workPosition3 = None, None, None
        workCityTowny1, workCityTowny2, workCityTowny3 = None, None, None
        workStatus1, workStatus2, workStatus3 = None, None, None

        for wk in range(0, len(workCompany)):
            if wk == 0: workCompany1, workIndustry1, workPosition1, workCityTowny1, workStatus1 = workCompany[wk], workIndustry[wk], workPosition[wk], workCityTowny[wk], workStatus[wk]
            if wk == 1: workCompany2, workIndustry2, workPosition2, workCityTowny2, workStatus2 = workCompany[wk], workIndustry[wk], workPosition[wk], workCityTowny[wk], workStatus[wk]
            if wk == 2: workCompany3, workIndustry3, workPosition3, workCityTowny3, workStatus3 = workCompany[wk], workIndustry[wk], workPosition[wk], workCityTowny[wk], workStatus[wk]

        try:
            eductionMajors = [x['major'] for x in i['education']]
            eductionSchoolName = [x['schoolCollegeName'] for x in i['education']]
            eductionGratuated = [x['graduated'] for x in i['education']]
        except: eductionMajors = eductionSchoolName = eductionGratuated = []

        eductionMajors1, eductionMajors2, eductionMajors3 = None, None, None
        eductionSchoolName1, eductionSchoolName2, eductionSchoolName3 = None, None, None
        eductionGratuated1, eductionGratuated2, eductionGratuated3 = None, None, None

        for wk in range(0, len(eductionMajors)):
            if wk == 0: eductionMajors1, eductionSchoolName1, eductionGratuated1 = eductionMajors[wk], eductionSchoolName[wk], eductionGratuated[wk]
            if wk == 1: eductionMajors2, eductionSchoolName2, eductionGratuated2 = eductionMajors[wk], eductionSchoolName[wk], eductionGratuated[wk]
            if wk == 2: eductionMajors3, eductionSchoolName3, eductionGratuated3 = eductionMajors[wk], eductionSchoolName[wk], eductionGratuated[wk]

        
        csvwriter.writerow([username, url,rating, track, developSkill, designSkill, dataScienceSkill, CopilitSkill, 
                            workCompany1, workIndustry1, workPosition1, workCityTowny1, workStatus1,
                            workCompany2, workIndustry2, workPosition2, workCityTowny2, workStatus2,
                            workCompany3, workIndustry3, workPosition3, workCityTowny3, workStatus3,
                            eductionMajors1, eductionSchoolName1, eductionGratuated1,
                            eductionMajors2, eductionSchoolName2, eductionGratuated2,
                            eductionMajors3, eductionSchoolName3, eductionGratuated3,
                            ])


print(f'CSV file saved to {csv_file_path}')

CSV file saved to c:\Users\PMLS\Desktop\topCoders\topCoders\SESSION_24_6_49\DEVELOPERS_STATS_24_6_54.csv
